In [ ]:
num_iters = 1000
env_num = 3
spec_num = 13

In [ ]:
from conformal.all_paths_conformal_pred import all_paths_conformal_pred
from conformal.bucketed_conformal_pred import bucketed_conformal_pred
from conformal.nonconformity_score_graph import DIRLCumRewardScoreGraph, DIRLTimeTakenScoreGraph
from spectrl.hierarchy.construction import adj_list_from_task_graph, automaton_graph_from_spec
from spectrl.hierarchy.reachability import HierarchicalPolicy, ConstrainedEnv
from spectrl.main.spec_compiler import ev, seq, choose, alw
from spectrl.util.io import parse_command_line_options, save_log_info, save_object
from spectrl.util.rl import print_performance, get_rollout
from spectrl.rl.ars import HyperParams

from spectrl.examples.rooms_envs import (
    GRID_PARAMS_LIST,
    MAX_TIMESTEPS,
    START_ROOM,
    FINAL_ROOM,
)
from spectrl.envs.rooms import RoomsEnv

import os


render = False
folder = ''
itno = -1

log_info = []

grid_params = GRID_PARAMS_LIST[env_num]

hyperparams = HyperParams(30, num_iters, 30, 15, 0.05, 0.3, 0.15)

print(
    "\n**** Learning Policy for Spec #{} in Env #{} ****".format(
        spec_num, env_num
    )
)

# Step 1: initialize system environment
system = RoomsEnv(grid_params, START_ROOM[env_num], FINAL_ROOM[env_num])

# Step 4: List of specs.
bottomright = (0, 3)
topleft = (3, 0)

# test specs
spec0 = ev(grid_params.in_room((1, 0)))  # 100 iterations works
spec1 = ev(grid_params.in_room((2, 0)))  # 200 iterations works
spec2 = ev(grid_params.in_room(topleft))  # 1000 iterations not enough
spec3 = alw(grid_params.avoid_center((1, 0)), ev(grid_params.in_room((2, 0))))  # 100 works
spec4 = seq(ev(grid_params.in_room((1, 0))), ev(grid_params.in_room(topleft)))  # 200 works
spec5 = seq(ev(grid_params.in_room((2, 0))), ev(grid_params.in_room((2, 2))))  # 200 works
spec6 = alw(grid_params.avoid_center((1, 0)), ev(
    grid_params.in_room(topleft)))  # 400 doesn't work
spec7 = alw(grid_params.avoid_center((2, 0)), ev(grid_params.in_room(topleft)))
spec8 = choose(ev(grid_params.in_room(topleft)),
                ev(grid_params.in_room(bottomright)))

spec9 = choose(alw(grid_params.avoid_center((1, 0)), ev(grid_params.in_room((2, 0)))),
                ev(grid_params.in_room((0, 2))))
spec10 = seq(spec9, ev(grid_params.in_room((2, 2))))
spec10part1 = choose(ev(grid_params.in_room((2, 1))),
                        ev(grid_params.in_room((3, 2))))
spec10part2 = seq(spec10part1,
                    ev(grid_params.in_room((3, 1))))
spec11 = seq(spec10, spec10part2)

spec11part1 = choose(ev(grid_params.in_room((1, 1))),
                        ev(grid_params.in_room((3, 3))))
spec11part2 = seq(spec11part1,
                    alw(grid_params.avoid_center((2, 3)), ev(grid_params.in_room((1, 3)))))
# spec11part2easy = ev(grid_params.in_room((1,3)))
spec12 = seq(spec11, spec11part2)

spec12part1 = choose(ev(grid_params.in_room((1, 1))),
                        ev(grid_params.in_room((0, 3))))
spec12part2 = seq(spec12part1, ev(grid_params.in_room((0, 1))))
spec13 = seq(spec12, spec12part2)

specs = [spec0, spec1, spec2, spec3, spec4, spec5, spec6,
            spec7, spec8, spec9, spec10, spec11, spec12, spec13]

# Step 3: construct abstract reachability graph
_, abstract_reach = automaton_graph_from_spec(specs[spec_num])
print("\n**** Abstract Graph ****")
abstract_reach.pretty_print()

# Step 5: Learn policy
path_policies = abstract_reach.learn_all_paths(
    system,
    hyperparams,
    res_model=None,
    max_steps=20,
    render=render,
    neg_inf=-100,
    safety_penalty=-1,
    num_samples=500,
)

adj_list = adj_list_from_task_graph(abstract_reach.abstract_graph)
terminal_vertices = [i for i in range(len(adj_list)) if i in adj_list[i]]


In [ ]:
time_taken_score_graph = DIRLTimeTakenScoreGraph(adj_list, path_policies)
n_samples = 2000
es = [0.2, 0.1, 0.05]
total_buckets = [5, 10, 20, 25, 50, 100]

data_time_taken = dict()
data_time_taken["metadata"] = {"es": es, "total_buckets": total_buckets, "scores": "cum-reward", "env": "16-rooms", "spec": spec_num, "n_samples": n_samples}

for e in es:
    e_data = dict()
    for buckets in total_buckets:
        bucket_data = dict()
        vbs = bucketed_conformal_pred(time_taken_score_graph, e, buckets, n_samples)
        min_path, min_path_scores = all_paths_conformal_pred(time_taken_score_graph, e, n_samples)
        vb = vbs.buckets[(terminal_vertices[0], buckets)]

        bucket_data["bucketed"] = {"path": vb.path, 
                                   "path_buckets": vb.path_buckets, 
                                   "path_score_quantiles": vb.path_score_quantiles, 
                                   "max_path_score_quantile": max(vb.path_score_quantiles)}
        bucket_data["all-paths"] = {"path": min_path, "min_path_scores": min_path_scores, "max_min_path_scores": max(min_path_scores)}
        e_data[buckets] = bucket_data
    data_time_taken[str(e)] = e_data

In [ ]:
import json

# Convert the Python object to a JSON string
json_data = json.dumps(data_time_taken, indent=2)

# Store the JSON string in a file
with open("conformal_experiments_data/16rooms-spec13-time-taken.json", "w") as json_file:
    json_file.write(json_data)

In [ ]:
cum_reward_score_graph = DIRLCumRewardScoreGraph(adj_list, path_policies)
n_samples = 2000
es = [0.2, 0.1, 0.05]
total_buckets = [5, 10, 20, 25, 50, 100]

data_time_taken = dict()
data_time_taken["metadata"] = {"es": es, "total_buckets": total_buckets, "scores": "cum-reward", "env": "9-rooms", "spec": spec_num, "n_samples": n_samples}

for e in es:
    e_data = dict()
    for buckets in total_buckets:
        bucket_data = dict()
        vbs = bucketed_conformal_pred(cum_reward_score_graph, e, buckets, n_samples)
        min_path, min_path_scores = all_paths_conformal_pred(cum_reward_score_graph, e, n_samples)
        vb = vbs.buckets[(terminal_vertices[0], buckets)]

        bucket_data["bucketed"] = {"path": vb.path, 
                                   "path_buckets": vb.path_buckets, 
                                   "path_score_quantiles": vb.path_score_quantiles, 
                                   "max_path_score_quantile": max(vb.path_score_quantiles)}
        bucket_data["all-paths"] = {"path": min_path, "min_path_scores": min_path_scores, "max_min_path_scores": max(min_path_scores)}
        e_data[buckets] = bucket_data
    data_time_taken[str(e)] = e_data

# Convert the Python object to a JSON string
json_data = json.dumps(data_time_taken, indent=2)

# Store the JSON string in a file
with open("conformal_experiments_data/16rooms-spec13-cum-reward.json", "w") as json_file:
    json_file.write(json_data)